<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Statistical information of VK groups - number of posts, tokens, unique words, and average post length.**



In [ ]:
import spacy
import pandas as pd
!python -m spacy download ru_core_news_sm
nlp = spacy.load("ru_core_news_sm")
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = stopwords.words("russian")
!pip install pymorphy3
import pymorphy3
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [4]:
posts = {
        'state': [
                'aparfenchikov',
                'minnazrk'
            ],

        'museum': [
                'rk_nationalmuseum',
                'olonmus',
                'museum_ptz',
                'echo_association',
                'domderevnivoknavolok'
            ],

        'language': [
                'satasanaa',
                'speechvepkar',
                'desyatiletieyazykovkarelia',
                'learning_karelian_language'
            ],

          'festival': [
                'omapajo',
                'kyykkakarjala',
                'etnodomrk',
                'tastykarjala'
                'senofest',
                'olongames',
                'kalevala_fest'
            ],

         'multifunctional': [
             'karjalankielenkodi',
             'mediacenter_periodika',
             'кarelian_flavor',
             'karjalanrahvahanliitto',
             'karelian_speakers'],
         }

In [5]:
def process_text(text, index, df):
    check_hash = False
    processed_parts = []
    for w in nltk.word_tokenize(text):
      if w == '#':
          check_hash = True
          continue
      if check_hash:
          check_hash = False
          continue
      w_tag = morph.parse(w.strip())[0].tag
      if  'Surn' in w_tag or 'Name' in w_tag or 'Patr' in w_tag:
        continue
      if w.isalpha() and w.lower() not in stop_words:
        if w.isupper() and len(w) <= 3:
            processed_parts.append(w)
        else:
            processed_parts.append(morph.parse(w.lower())[0].normal_form)

    result = ' '.join(processed_parts)
    df.at[index, 'tokens'] = str(result)

In [6]:
urls = []
names = []
groups_posts = []
tokens = []
unique_words = []
mid_len_post = []
for key in posts.keys():
    for name in posts[key]:
      url = f'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/{name}.csv'
      urls.append(url)
      names.append(name)
      t = 0
      dict_lem = {}
      try:
        df = pd.read_csv(url, usecols = ['text'],encoding='utf-8')
      except:
        print(url)
      df = df[df['text'].apply(lambda x: isinstance(x, str))]
      groups_posts.append(len(df))
      for index, row in df.iterrows():
        process_text(row['text'],index, df)
      df.to_excel(url.split('/')[-1].rstrip('.csv')+'_df_tokens.xlsx', index=False)
      df_tokens = df['tokens']
      for row in df_tokens:
        row_split = row.split()

        for lemma in row_split:
          if dict_lem.get(lemma, 0) == 0:
              dict_lem[lemma] = 1
          else:
              dict_lem[lemma] += 1
      data = df['tokens'].tolist()
      for d in data:
        if type(d) == str:
          t += len(d.split())
      mid_len_post.append(t // len(df))
      dict_lem = dict(sorted(dict_lem.items(), key=lambda item: item[1], reverse=True))
      tokens.append(t)
      unique_words.append(len(dict_lem))
      t = 0


<ipython-input-5-05c5622d286e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'tokens'] = str(result)


https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/tastykarjalasenofest.csv
https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/кarelian_flavor.csv


In [7]:
df

,text,tokens
0,Verkkoversio on osoitteessa http://kaino.kotus...,verkkoversio on osoitteessa http ссылка словар...
1,Kuvahaisie Karjalasta.,kuvahaisie karjalasta
2,KARJALAN HEIMO Правление отделения Карельског...,karjalan heimo правление отделение карельский ...
3,Piiterissä.,piiterissä
4,Karjalan kielestä sen verran jotta kieli on ni...,karjalan kielestä sen verran jotta kieli on ni...
...,...,...
4505,Terveh)) on hengis kieli tuattoloin),terveh on hengis kieli tuattoloin
4506,A vot! Tässäi mie.,A vot tässäi mie
4507,"Hyvä kysymys! Semmoset älvisköt kun olemma, em...",hyvä kysymys semmoset älvisköt kun olemma emmä...
4508,Носители карельского! А почему группа российск...,носитель карельский почему группа российскоязы...


In [8]:
d = {'Группа ВК': names, 'URL': urls, 'Количество постов': groups_posts, 'Количество токенов': tokens, 'Количество уникальных слов': unique_words, 'Средняя длина поста в словах': mid_len_post}
df = pd.DataFrame(data=d)
df.to_csv('statistics.csv', index=False, encoding='utf-8')

In [9]:
df

,Группа ВК,URL,Количество постов,Количество токенов,Количество уникальных слов,Средняя длина поста в словах
0,aparfenchikov,https://raw.githubusercontent.com/componavt/sn...,8808,721527,22806,81
1,minnazrk,https://raw.githubusercontent.com/componavt/sn...,6052,352167,14981,58
2,rk_nationalmuseum,https://raw.githubusercontent.com/componavt/sn...,6714,346067,18135,51
3,olonmus,https://raw.githubusercontent.com/componavt/sn...,2122,145900,15126,68
4,museum_ptz,https://raw.githubusercontent.com/componavt/sn...,1198,67884,7619,56
5,echo_association,https://raw.githubusercontent.com/componavt/sn...,2040,95771,10268,46
6,domderevnivoknavolok,https://raw.githubusercontent.com/componavt/sn...,2166,66822,10357,30
7,satasanaa,https://raw.githubusercontent.com/componavt/sn...,790,48353,4975,61
8,speechvepkar,https://raw.githubusercontent.com/componavt/sn...,24,1312,422,54
9,desyatiletieyazykovkarelia,https://raw.githubusercontent.com/componavt/sn...,964,49850,9396,51
